In [3]:
import pandas as pd
import numpy as np
import oracledb
import openpyxl
from openpyxl.styles import Font, Alignment, PatternFill, numbers  # Import the Font and PatternFill class
from datetime import datetime

# Get today's date and format it
today = datetime.today()
todays_date = today.strftime('%Y-%m-%d')

In [4]:
# Establish the database connection
connection = oracledb.connect(user='usernamehere',
                              password='pswdhere',
                              dsn=oracledb.makedsn('hostnamehere', '1234', service_name='DATABASE_NAME_HERE'))
cursor = connection.cursor()

query = """
WITH base_query AS (
    SELECT 
        b.udc_category,
        b.udc_category_descr,
        a.loc,
        b.item,
        s.vendornum,
        b.vendorname,
        b.DESCR,
        b.DESCR2,
        b.UDC_WEIGHT,
        b.UDC_CUBE,
        a.startdate AS "STARTDATE YYYY/MM/DD",
        s.message1,
        s.message2,
        s.message3,
        s.message4,
        c.udc_order_review_cal,
        s.iflag,
        s.rflag,
        s.udc_omnistatus,
        s.udc_surgitrack,
        s.buyer,
        b.udc_dollar AS "Unit Cost",
        b.puruomconvfact "PUOM Conversion Factor",
        b.baseuom "Base UOM",
        s.UDC_4weekfcst AS "4wk Forecast @ DC",
        s.udc_totalpastduepo AS "Total Qty Past Due", 
        covdur/1440 AS "Days Coverage Duration",
        projavail,
        projavail*b.udc_dollar AS "PROJAVAIL Cost",
        s.oh AS "Qty On Hand",
        s.oh*b.udc_dollar AS "Total On Hand Cost"
    FROM 
        SCPDBINSTANCE.skuprojstatic a
        INNER JOIN SCPDBINSTANCE.item b ON a.item = b.item
        INNER JOIN SCPDBINSTANCE.sku s ON a.item = s.item AND a.loc = s.loc
        INNER JOIN SCPDBINSTANCE.udt_network c ON c.udc_vendornum = s.vendornum AND c.udc_vendor_line_code = s.vendor_line_code AND c.udc_loc = s.loc
    WHERE 
        a.item NOT LIKE '8%' -- remove COI Items
        AND s.oh >= 1
        AND a.projavail > 0
        AND a.covdur/1440 >= 30 -- anything where DOH is over 30 days
        AND trunc(a.startdate) = trunc(sysdate)+45 -- used to look 45 days out. Is normally 45 days out
        AND (a.item,a.loc) NOT IN (
            SELECT item,dest
            FROM SCPDBINSTANCE.planarriv
            WHERE trunc(schedarrivdate) <= trunc(sysdate)+45 -- this is normally 45 days out
        )
),
planarriv_query AS (
    SELECT 
        a.item AS pa_item, 
        dest,
        SUM(qty) AS planarriv_qty
    FROM 
        SCPDBINSTANCE.planarriv a
        INNER JOIN SCPDBINSTANCE.item b ON a.item = b.item
    WHERE 
        trunc(schedarrivdate) <= trunc(sysdate)+45 -- this query pulls what the network needs
    GROUP BY 
        a.item,
        dest
)
SELECT * 
FROM (
    SELECT * 
    FROM base_query s 
    LEFT JOIN planarriv_query p ON s.item = p.pa_item
)
PIVOT (
    SUM(NVL(planarriv_qty,0)) 
    FOR dest IN ('DIV 03','DIV 08','DIV 13','DIV 14','DIV 16','DIV 20','DIV 21','DIV 30','DIV 37','DIV 41','DIV 42','DIV 44',
    'DIV 45','DIV 48','DIV 49','DIV 50','DIV 51','DIV 53','DIV 56','DIV 58','DIV 59','DIV 60','DIV 61','DIV 64','DIV 65',
    'DIV 66','DIV 67','DIV 68','DIV 69','DIV 70','DIV 71','DIV 78','DIV 80','DIV 82','DIV 84','DIV 85','DIV 87','DIV 89',
    'DIV 90','DIV 91','DIV 92','DIV 93','DIV 94','DIV 96','DIV 98')
)
"""

# Execute a query
cursor.execute(query)

# Fetch all rows
rows = cursor.fetchall()

# Get the column names
column_names = [column[0] for column in cursor.description]

data_excess = pd.DataFrame(rows, columns=column_names)

# Close database connection
connection.close()

# Remove single quotations around the DIV columns which are a result of the pivot done in the SQL query
data_excess.columns = [col.replace("'", "") for col in data_excess.columns]

In [5]:
### Data formatting and manipulation

# Make a copy dataframe of raw data pulled from BY
df_excess = data_excess

# Get list of columns that start with 'DIV'
div_columns = [col for col in df_excess.columns if col.startswith("DIV")]

# Convert the data in 'DIV' columns to float
for col in div_columns:
    df_excess[col] = df_excess[col].astype(float)

# Format startdate column
df_excess['STARTDATE YYYY/MM/DD'] = pd.to_datetime(df_excess['STARTDATE YYYY/MM/DD']).dt.strftime('%Y/%m/%d')



### Create columns based on DC data

df_excess['Avail to Move Qty'] = np.where(df_excess['Qty On Hand'] >= df_excess['PROJAVAIL'], 
                                          np.where(df_excess['PROJAVAIL'] < 1, 0, np.floor(df_excess['PROJAVAIL'])), 0)

df_excess['Avail to Move $'] = df_excess['Unit Cost'] * df_excess['Avail to Move Qty']

df_excess['Total Needed at DCs --->'] = df_excess[div_columns].sum(axis=1)

df_excess['Potential to Move Qty'] = np.where(df_excess['Avail to Move Qty'] == 0, 
                                       0, 
                                       np.where(df_excess['Total Needed at DCs --->'] - df_excess['Avail to Move Qty'] > 0, 
                                                df_excess['Avail to Move Qty'], 
                                                df_excess['Total Needed at DCs --->']))

df_excess['Potential to Move $'] = df_excess['Potential to Move Qty'] * df_excess['Unit Cost']

df_excess['Potential to Move Cube'] = df_excess['Potential to Move Qty'] * df_excess['UDC_CUBE']

df_excess['Potential to Move Weight'] = df_excess['Potential to Move Qty'] * df_excess['UDC_WEIGHT']

df_excess = df_excess.sort_values(by='Potential to Move $', ascending=False)

df_excess['Qty Delta of Avail vs Potential'] = df_excess['Avail to Move Qty'] - df_excess['Potential to Move Qty']

df_excess['$ Delta of Avail vs Potential'] = df_excess['Avail to Move $'] - df_excess['Potential to Move $']

# Function to apply the Excel formula logic
def coverage_duration_buckets(value):
    if value < 60:
        return "1) < 60"
    elif value < 120:
        return "2) 61 - 120"
    elif value < 180:
        return "3) 120 - 180"
    elif value < 270:
        return "4) 180 - 270"
    elif value >= 270:
        return "5) 270+"
    else:
        return "NA"

# Apply the function to the 'Days Coverage Duration' column
df_excess['Days Coverage Duration Buckets'] = df_excess['Days Coverage Duration'].apply(coverage_duration_buckets)

# Create a DC-SKU key column
df_excess['KEY: OM DC + OM SKU'] = df_excess['LOC'].str[-2:] + df_excess['ITEM']

# Reorder columns
df_excess = df_excess.reindex(columns=['UDC_CATEGORY', 'UDC_CATEGORY_DESCR',
                                       'LOC', 'ITEM', 'VENDORNUM', 'VENDORNAME',
                                       'DESCR', 'DESCR2', 'STARTDATE YYYY/MM/DD', 'MESSAGE1', 'MESSAGE2', 'MESSAGE3', 'MESSAGE4',
                                       'UDC_CUBE', 'UDC_WEIGHT', 'UDC_ORDER_REVIEW_CAL', 'IFLAG', 'RFLAG', 'UDC_OMNISTATUS', 'UDC_SURGITRACK', 'BUYER',
                                       'Unit Cost', 'PUOM Conversion Factor', 'Base UOM',
                                       '4wk Forecast @ DC', 'Total Qty Past Due', 'Days Coverage Duration', 'PROJAVAIL', 'PROJAVAIL Cost',
                                       'Qty On Hand', 'Total On Hand Cost', 'Days Coverage Duration Buckets', 'KEY: OM DC + OM SKU',
                                       'Avail to Move Qty', 'Avail to Move $', 'Potential to Move Qty', 'Potential to Move $', 'Potential to Move Cube', 'Potential to Move Weight',
                                       'Qty Delta of Avail vs Potential', '$ Delta of Avail vs Potential',
                                       'Approved Move Qty', 'Reason for NOT Moving', '$ Moved', # This line of code also creates these empty columns used by Mary McKenna
                                       'Total Needed at DCs --->',
                                       'DIV 03', 'DIV 08', 'DIV 13', 'DIV 14', 'DIV 16', 'DIV 20', 'DIV 21', 'DIV 30', 'DIV 37', 'DIV 41', 'DIV 42',
                                       'DIV 44', 'DIV 45', 'DIV 48', 'DIV 49', 'DIV 50', 'DIV 51', 'DIV 53', 'DIV 56', 'DIV 58', 'DIV 59', 'DIV 60',
                                       'DIV 61', 'DIV 64', 'DIV 65', 'DIV 66', 'DIV 67', 'DIV 68', 'DIV 69', 'DIV 70', 'DIV 71', 'DIV 78', 'DIV 80',
                                       'DIV 82', 'DIV 84', 'DIV 85', 'DIV 87', 'DIV 89', 'DIV 90', 'DIV 91', 'DIV 92', 'DIV 93', 'DIV 94', 'DIV 96', 'DIV 98'
                                       ])



### Create tab for data dictionary definitions
data = {
    'Description': ['Columns starting with DIV:', 'Avail to Move Qty:', 'Potential to Move Qty:', 'Coverage Duration Days:'],
    'Formula': ['', '`=IF(Qty On Hand>=PROJAVAIL,IF(PROJAVAIL<1,0,ROUNDDOWN(PROJAVAIL,0.1)),0)',
                '`=IF(Avail to Move Qty=0,0,(IF((Total Needed at DCs-Avail to Move Qty)>0,(Avail to Move Qty),Total Needed at DCs)))', ''],
    'Explanation': [
        "Show the qtys that the corresponding DC could potentially absorb, given the DC's forecast for the item.",
        "Total qty available that could be moved to the corresponding DC on the right.",
        "Total qty that could actually be moved to the corresponding DC on the right.",
        "This report is saying that 45 days out from now, this is the COVDUR Days for this item @ its DC"
    ]
}

df_datadictionary = pd.DataFrame(data)



### Create Pivot reports

# Filter the dataframe based on 'IFLAG' and 'LOC'
# filtered_df = df_excess[(df_excess['IFLAG'] == 'I') & (~df_excess['LOC'].isin(['DIV 03', 'DIV 07', 'DIV 12', 'DIV 42', 'DIV 61']))]

# Create a pivot table of $ potential to move by days on hand
pivot_potentialtomovebyDOHbuckets = pd.pivot_table(df_excess, values='Potential to Move $', index='Days Coverage Duration Buckets', aggfunc='sum').reset_index()

# Calculate the total sum of 'Potential to Move $'
total_potential_to_move = df_excess['Potential to Move $'].sum()

# Append the total sum to the pivot table
pivot_potentialtomovebyDOHbuckets.loc['Total'] = ['Total', total_potential_to_move]

# Create another pivot table, potential to move by DC
pivot_potentialtomovebydc = pd.pivot_table(df_excess, values='Potential to Move $', index='LOC', aggfunc='sum').reset_index()

In [6]:
print("Number of records in this week's data pull: ", len(df_excess))
print()
print("Total $ Potential to Move this week: $", total_potential_to_move)

Number of records in this week's data pull:  88635

Total $ Potential to Move this week: $ 16812229.9166


In [7]:
# Create an Excel writer
excessFilewriter = pd.ExcelWriter(f'Excess and Planned Arrivals {todays_date}.xlsx', engine='xlsxwriter')

# Write resulting dataframes to separate tabs on Excel workbook
df_excess.to_excel(excessFilewriter, sheet_name='Excess Report', index=False)
pivot_potentialtomovebyDOHbuckets.to_excel(excessFilewriter, sheet_name='Potential To Move DOH Buckets', index=False)
pivot_potentialtomovebydc.to_excel(excessFilewriter, sheet_name='Potential To Move by DC', index=False)
df_datadictionary.to_excel(excessFilewriter, sheet_name='Data Dictionary', index=False)

# Close the Excel writer
excessFilewriter.close()



### Excel Spreadsheet formatting using Openpyxl

# Load your existing workbook
workbook = openpyxl.load_workbook(f'Excess and Planned Arrivals {todays_date}.xlsx')

# Create formatting for headers
header_font = openpyxl.styles.NamedStyle(name='header_font')
header_font.font = Font(name='Calibri', bold=True)
header_font.alignment = Alignment(wrap_text=True, horizontal='center', vertical='center')

# Apply font to all cells in each worksheet
for sheet_name in workbook.sheetnames:
    sheet = workbook[sheet_name]

    # Apply text wrap and center alignment to header row
    for cell in sheet[1]:
        cell.style = header_font
        sheet.column_dimensions[cell.column_letter].alignment = Alignment(horizontal='left')  # Left align

    # Add filter to the header row and freeze top row
    sheet.auto_filter.ref = sheet.dimensions
    sheet.freeze_panes = 'A2'

# Get the 'Excess Report' worksheet
excess_report_sheet = workbook['Excess Report']

# Define the columns to format as currency in the 'excess_report_sheet' workbook
# Get the column indices for the columns to format
columns_to_currencyformat = ['Unit Cost', 'PROJAVAIL Cost', 'On Hand Cost', 'Total On Hand Cost', 'Potential to Move $', '$ Delta of Avail vs Potential', 'Avail to Move $']
column_indices = [excess_report_sheet[1].index(cell) + 1 for cell in excess_report_sheet[1] if cell.value in columns_to_currencyformat]

# Apply currency format to columns in the Excess report sheet
for col_idx in column_indices:
    col_letter = openpyxl.utils.get_column_letter(col_idx)
    for cell in excess_report_sheet[col_letter]:
        cell.number_format = numbers.FORMAT_CURRENCY_USD_SIMPLE

# Format column B in the 'Potential To Move DOH Buckets' and 'Potential To Move by DC' tabs
for sheet_name in ['Potential To Move DOH Buckets', 'Potential To Move by DC']:
    for cell in workbook[sheet_name]['B']:
        cell.number_format = numbers.FORMAT_CURRENCY_USD_SIMPLE

# Create function to format widths of columns and color of a few specific columns
def set_column_widths(sheet, column_widths):
    for col in sheet.columns:
        column_letter = col[0].column_letter
        if column_letter in column_widths:
            sheet.column_dimensions[column_letter].width = column_widths[column_letter]
        elif column_letter > 'AO':
            sheet.column_dimensions[column_letter].width = 5

def set_background_colors(sheet, color_mappings):
    for (start_col, end_col), color in color_mappings.items():
        for row in sheet.iter_rows(min_col=start_col, max_col=end_col):
            for cell in row:
                cell.fill = PatternFill(start_color=color, end_color=color, fill_type='solid')

# Define the desired column widths (adjust as needed)
column_widths = {
    'A': 8.11, 'B': 15, 'C': 8, 'D': 15, 'E': 7.33, 'F': 18, 'G': 35,
    'H': 25, 'I': 11, 'J': 24, 'K': 24, 'L': 24, 'M': 24, 'N': 7, 'O': 7, 'P': 7,
    'Q': 7, 'R': 7, 'S': 7, 'T':7, 'U':7, 'V': 10, 'W': 8, 'X': 6, 'Y': 8, 'Z': 6, 'AA': 9, 'AB': 6,
    'AC': 12, 'AD': 6, 'AE': 14, 'AF': 12, 'AG': 13, 'AH': 11, 'AI': 15, 'AJ': 11,
    'AK': 15, 'AL': 11, 'AM': 11, 'AN': 11, 'AO': 15, 'AP': 10, 'AQ': 10, 'AR': 10, 'AS': 10
}

# Set background colors for specific columns
color_mappings = {
    (34, 35): 'B8CCE4',
    (36, 39): 'C4D79B',
    (42, 44): 'FFE699',
    (45, 45): 'B8CCE4',
}

# Adjust column widths and set background colors for each sheet
for sheet_name in workbook.sheetnames:
    sheet = workbook[sheet_name]
    set_column_widths(sheet, column_widths)
    set_background_colors(sheet, color_mappings)

# Define the desired column widths for the other worksheets
sheet_column_widths = {
    'Potential To Move DOH Buckets': {'A': 11, 'B': 15},
    'Potential To Move by DC': {'A': 11, 'B': 15},
    'Data Dictionary': {'A': 25, 'B': 25}
}

# Adjust column widths for each sheet
for sheet_name, column_widths in sheet_column_widths.items():
    sheet = workbook[sheet_name]
    for column_letter, width in column_widths.items():
        sheet.column_dimensions[column_letter].width = width

# Save the modified workbook
workbook.save(f'Excess and Planned Arrivals {todays_date}.xlsx')